Objectives:

1. Make use of Google Colab’s free GPU to train a HuggingFace model
2. Follow the documentation from start to finish
Be able to answer questions about each piece of code during the interview
3. Demonstrate fine-tuning using the sample dataset provided.
4. Bonus points: Use another audio classification dataset to perform fine-tuning



Set up 
1. install the libraries
  - dataset: lib where we can get data
  - transformers: hugging face library
  - librosa: package to load audio files
2. notebook log in 
3. telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts.

source: https://github.com/huggingface/notebooks/blob/main/examples/audio_classification.ipynb


In [1]:
!pip install datasets
!pip install transformers
!pip install librosa 

!apt install git-lfs
! pip install pynvml 


#install the needed libraries

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 9.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 83.8 MB/s eta 0:00:

In [2]:
from transformers.utils import send_example_telemetry

In [5]:
from huggingface_hub import notebook_login
from datasets import load_dataset, load_metric
from transformers import AutoFeatureExtractor,AutoModelForAudioClassification, TrainingArguments, Trainer, AutoModelForAudioClassification, AutoFeatureExtractor





In [6]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [7]:

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [9]:
send_example_telemetry("text_classification_notebook", framework="pytorch")

## Model definition

### ```facebook/wav2vec2-base```
- base model pretrained on the 100k unlabeled subset of VoxPopuli corpus.
-  In order to use this model speech recognition, a tokenizer should be created and the model should be fine-tuned on labeled text data


In [11]:
send_emodel_checkpoint = "facebook/wav2vec2-base"
batch_size = 8


## Loading the dataset

Background:
- similar to the text classfication, we get the dataset and metric from dataset library

Sample Dataset description: 
- SUPERB uses the widely used Speech Commands
- Task: Keyword Spotting, the model has a version with a Sequence Classification head
- [source](https://huggingface.co/datasets/superb)

My Dataset Description
- description: This is a set of one-second .wav audio files, each containing a single spoken English word or background noise
- configuration v0.02: was released on April 11th 2018 and contains 105,829 audio files.
- Task: Keyword Spotting to detect preregistered keywords by classifying utterances
- dataset size
  - train: 84848
  - validation: 9982	
  - test: 4890

[source](https://huggingface.co/datasets/speech_commands)





In [12]:
dataset = load_dataset("superb", "ks")
metric = load_metric("accuracy")




Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/51094 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6798 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3081 [00:00<?, ? examples/s]

Dataset superb downloaded and prepared to /root/.cache/huggingface/datasets/superb/ks/1.9.0/b8183f71eabe8c559d7f3f528ab37a6a21ad1ee088fd3423574cecad8b3ec67e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-12-05f5d4a804d2>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [13]:
#speech command dataset
sc_ds = load_dataset("speech_commands",'v0.02')

Generating train split:   0%|          | 0/84848 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9982 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4890 [00:00<?, ? examples/s]

Dataset speech_commands downloaded and prepared to /root/.cache/huggingface/datasets/speech_commands/v0.02/0.2.0/ba3d9a6cf49aa1313c51abe16b59203451482ccb9fee6d23c94fecabf3e206da. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

### Check the labels
- ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'bed', 'bird', 'cat', 'dog', 'happy', 'house', 'marvin', 'sheila', 'tree', 'wow', 'backward', 'forward', 'follow', 'learn', 'visual', '_silence_']
- 36 number of classes




In [14]:
#show dataset dictionary
print(sc_ds)

#show one sample
print(sc_ds["train"][32])

#As you can see, the label field is not an actual string label.
# By default the ClassLabel fields are encoded into integers for convenience:


print(sc_ds["train"].features["label"])
print(len(sc_ds["train"].features["label"].names))

# print((sc_ds["train"].features["label"])

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'label', 'is_unknown', 'speaker_id', 'utterance_id'],
        num_rows: 84848
    })
    validation: Dataset({
        features: ['file', 'audio', 'label', 'is_unknown', 'speaker_id', 'utterance_id'],
        num_rows: 9982
    })
    test: Dataset({
        features: ['file', 'audio', 'label', 'is_unknown', 'speaker_id', 'utterance_id'],
        num_rows: 4890
    })
})
{'file': 'backward/d394ef8e_nohash_3.wav', 'audio': {'path': 'backward/d394ef8e_nohash_3.wav', 'array': array([ 0.00021362,  0.00027466,  0.00018311, ..., -0.00048828,
       -0.00024414, -0.00024414]), 'sampling_rate': 16000}, 'label': 30, 'is_unknown': True, 'speaker_id': 'd394ef8e', 'utterance_id': 3}
ClassLabel(names=['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'bed', 'bird', 'cat', 'dog', 'happy', 'house', 'marvin', 'sheila', 'tree', 'wow'

create the following
- label2id: key is the label
- id2label: key is the id

we need this as an argument for the model

In [15]:
labels = sc_ds["train"].features["label"].names

label2id, id2label = dict(), dict()

for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["35"]

'_silence_'

side note the sampling rate is set to 16kHz which is what Wav2Vec2

## Preprocessing the data


instatiate feature_extractor to normalize and transform the data that's readable by the model

In [16]:

model_checkpoint = "facebook/wav2vec2-base"
batch_size = 8

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)

feature_extractor

/usr/local/lib/python3.9/dist-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

1. Create a function to preprocess the faetures
  - truncation=True
  - maximum sample length ( so we can identify the silence)
  - methodology
    - audio_arrays: extract the audio an turn into an array 
    -  feed the audio array into feature_extractor
2. Map the feature extractor to the sc_ds (dataset)

In [17]:
# dont need to run this cause all the samples are 1 second long
#max_duration = 1.0  # seconds
max_duration = 1.0
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
    return inputs

preprocess_function(sc_ds['train'][:3])


{'input_values': [array([ 0.00184695,  0.00184695,  0.00184695, ..., -0.01184682,
       -0.01527027, -0.01184682], dtype=float32), array([-0.00026575,  0.00522273,  0.00583256, ..., -0.00534767,
       -0.00270507, -0.00473784], dtype=float32), array([0.00399843, 0.00399843, 0.01274386, ..., 0.07770992, 0.09686658,
       0.11768904], dtype=float32)]}

In [18]:
#encoded speech comands dataset
enc_sc_ds = sc_ds.map(preprocess_function, remove_columns=["audio", "file"], batched=True)
enc_sc_ds

Map:   0%|          | 0/84848 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/feature_extraction_utils.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


Map:   0%|          | 0/9982 [00:00<?, ? examples/s]

Map:   0%|          | 0/4890 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'is_unknown', 'speaker_id', 'utterance_id', 'input_values'],
        num_rows: 84848
    })
    validation: Dataset({
        features: ['label', 'is_unknown', 'speaker_id', 'utterance_id', 'input_values'],
        num_rows: 9982
    })
    test: Dataset({
        features: ['label', 'is_unknown', 'speaker_id', 'utterance_id', 'input_values'],
        num_rows: 4890
    })
})

### Train the model
- we define the initial model checkpoint which is  was "facebook/wav2vec2-base"
- we need to pass label2id and id2label so the moedel knowsl how to decode the output


In [19]:


total_labels = len(id2label)

init_model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint, 
    num_labels=total_labels,
    label2id=label2id,
    id2label=id2label,
).to("cuda")

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForSequenceClassification: ['project_q.bias', 'quantizer.weight_proj.bias', 'project_hid.bias', 'project_q.weight', 'project_hid.weight', 'quantizer.weight_proj.weight', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['projector.bias', 'projector.weight', 'classifier.

error explenation
- Removing the head used to pretrain the model on an unsupervised Vector Quantization objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

- the args is similar to text classification

In [20]:


model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-ks",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)


import numpy as np


# Next, we need to define a function for how to compute the metrics from the predictions, which will just use the metric we loaded earlier. The only preprocessing we have to do is to take the argmax of our predicted logits
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

Then we just need to pass all of this along with our datasets to the Trainer:



In [21]:
trainer = Trainer(
    init_model,
    args,
    train_dataset=enc_sc_ds["train"],
    eval_dataset=enc_sc_ds["validation"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/Dc26/wav2vec2-base-finetuned-ks into local empty directory.


In [22]:
result = trainer.train()
# the fine tunning takes 3 hours
# reco look for the reason why training takes so long 
print_summary(result)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,0.640800,0.239874,0.960930
2,0.411300,0.128373,0.968644
2,0.301400,0.114351,0.970647
4,0.291400,0.096195,0.975957
4,0.162100,0.088022,0.977960


Time: 13264.36
Samples/second: 31.98
GPU memory occupied: 3261 MB.


In [23]:
trainer.evaluate()

{'eval_loss': 0.08802212029695511,
 'eval_accuracy': 0.9779603285914646,
 'eval_runtime': 123.8795,
 'eval_samples_per_second': 80.578,
 'eval_steps_per_second': 10.074,
 'epoch': 5.0}

In [24]:
trainer.push_to_hub()

Upload file runs/Mar29_14-20-33_42a8299ce90b/events.out.tfevents.1680099639.42a8299ce90b.714.0:  15%|#5       …

Upload file runs/Mar29_14-20-33_42a8299ce90b/events.out.tfevents.1680113027.42a8299ce90b.714.2: 100%|#########…

To https://huggingface.co/Dc26/wav2vec2-base-finetuned-ks
   37fdba4..e454be7  main -> main

   37fdba4..e454be7  main -> main

To https://huggingface.co/Dc26/wav2vec2-base-finetuned-ks
   e454be7..b95b429  main -> main

   e454be7..b95b429  main -> main



'https://huggingface.co/Dc26/wav2vec2-base-finetuned-ks/commit/e454be75039d14ec936a770fc47a9f783c15e539'

In [25]:
model_name

'wav2vec2-base'

In [27]:
# from transformers import AutoModelForAudioClassification, AutoFeatureExtractor
# AutoModelForSequenceClassification.from_pretrained("Dc26/"+model_name)


feature_extractor = AutoFeatureExtractor.from_pretrained("Dc26/"+model_name+"-finetuned-ks")
model = AutoModelForAudioClassification.from_pretrained("Dc26/"+model_name+"-finetuned-ks")

Had less documentation, since text classification and audio classificationa are similar